In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
dic = pd.read_csv('data_dict.csv')
dic = dic.drop(['NOTES','SOURCE'],axis=1)
dic = dic.set_index('VARIABLE NAME')
dic

,NAME OF DATA ELEMENT,dev-category,developer-friendly name,API data type,VALUE,LABEL
VARIABLE NAME,,,,,,
UNITID,Unit ID for institution,root,id,integer,NaN,NaN
OPEID,8-digit OPE ID for institution,root,ope8_id,string,NaN,NaN
OPEID6,6-digit OPE ID for institution,root,ope6_id,string,NaN,NaN
INSTNM,Institution name,school,name,autocomplete,NaN,NaN
CITY,City,school,city,autocomplete,NaN,NaN
...,...,...,...,...,...,...
OMENRUP_FULLTIME_POOLED_SUPP,Percentage of full-time students that did not ...,completion,outcome_percentage_suppressed.full_time.8yr.un...,float,NaN,NaN
OMENRYP_PARTTIME_POOLED_SUPP,Percentage of part-time students that did not ...,completion,outcome_percentage_suppressed.part_time.8yr.st...,float,NaN,NaN
OMENRAP_PARTTIME_POOLED_SUPP,Percentage of part-time students that did not ...,completion,outcome_percentage_suppressed.part_time.8yr.tr...,float,NaN,NaN


In [50]:
cols = [
    "UNITID",
    "INSTNM",
    "CITY",
    "STABBR",
    "ACCREDAGENCY",
    "HCM2",
    "MAIN",
    "NUMBRANCH",
    "PREDDEG",
    "HIGHDEG",
    "CONTROL",
    "REGION",
    "LOCALE",
    "CCBASIC",
    "CCUGPROF",
    "CCSIZSET",
    "HBCU",
    "PBI",
    "ANNHI",
    "TRIBAL",
    "AANAPII",
    "HSI",
    "MENONLY",
    "WOMENONLY",
    "RELAFFIL",
    "ADM_RATE",
    "ADM_RATE_ALL",
    "SATVR25",
    "SATVR75",
    "SATMT25",
    "SATMT75",
    "SATWR25",
    "SATWR75",
    "SATVRMID",
    "SATMTMID",
    "SATWRMID",
    "ACTCM25",
    "ACTCM75",
    "ACTEN25",
    "ACTEN75",
    "ACTMT25",
    "ACTMT75",
    "ACTWR25",
    "ACTWR75",
    "ACTCMMID",
    "ACTENMID",
    "ACTMTMID",
    "ACTWRMID",
    "SAT_AVG",
    "SAT_AVG_ALL",
    "PCIP01",
    "PCIP03",
    "PCIP04",
    "PCIP05",
    "PCIP09",
    "PCIP10",
    "PCIP11",
    "PCIP12",
    "PCIP13",
    "PCIP14",
    "PCIP15",
    "PCIP16",
    "PCIP19",
    "PCIP22",
    "PCIP23",
    "PCIP24",
    "PCIP25",
    "PCIP26",
    "PCIP27",
    "PCIP29",
    "PCIP30",
    "PCIP31",
    "PCIP38",
    "PCIP39",
    "PCIP40",
    "PCIP41",
    "PCIP42",
    "PCIP43",
    "PCIP44",
    "PCIP45",
    "PCIP46",
    "PCIP47",
    "PCIP48",
    "PCIP49",
    "PCIP50",
    "PCIP51",
    "PCIP52",
    "PCIP54",
    "DISTANCEONLY",
    "UGDS",
    "UG",
    "UGDS_WHITE",
    "UGDS_BLACK",
    "UGDS_HISP",
    "UGDS_ASIAN",
    "UGDS_AIAN",
    "UGDS_NHPI",
    "UGDS_2MOR",
    "UGDS_NRA",
    "UGDS_UNKN",
    "UGDS_WHITENH",
    "UGDS_BLACKNH",
    "UGDS_API",
    "UGDS_AIANOLD",
    "UGDS_HISPOLD",
    "UG_NRA",
    "UG_UNKN",
    "UG_WHITENH",
    "UG_BLACKNH",
    "UG_API",
    "UG_AIANOLD",
    "UG_HISPOLD",
    "PPTUG_EF",
    "PPTUG_EF2",
    "CURROPER",
    "NPT4_PUB",
    "NPT4_PRIV",
    "NPT4_PROG",
    "NUM4_PUB",
    "NUM4_PRIV",
    "NUM4_PROG",
    "NUM4_OTHER",
    "COSTT4_A",
    "COSTT4_P",
    "TUITIONFEE_IN",
    "TUITIONFEE_OUT",
    "TUITIONFEE_PROG",
    "TUITFTE",
    "INEXPFTE",
    "AVGFACSAL",
    "PFTFAC",
    "PCTPELL",
    "PCTFLOAN",
    "UG25ABV",
    "COUNT_NWNE_P10",
    "COUNT_WNE_P10",
    "MN_EARN_WNE_P10",
    "MD_EARN_WNE_P10",
    "COUNT_NWNE_P10",
    "COUNT_WNE_P10",
    "MN_EARN_WNE_P10",
    "MD_EARN_WNE_P10",
    "COUNT_NWNE_P6",
    "COUNT_WNE_P6",
    "MN_EARN_WNE_P6",
    "MD_EARN_WNE_P6",
]

In [51]:
df_cols = pd.DataFrame(cols)
df_cols.columns = ["variable"]
df_cols["category"] = df_cols.variable.apply(lambda x: dic.loc[x, "dev-category"])
df_cols["type"] = df_cols.variable.apply(lambda x: dic.loc[x, "API data type"])
df_cols["friendly_name"] = df_cols.variable.apply(
    lambda x: dic.loc[x, "developer-friendly name"]
)
df_cols["explanation"] = df_cols.variable.apply(
    lambda x: dic.loc[x, "NAME OF DATA ELEMENT"]
)
df_cols.head()

,variable,category,type,friendly_name,explanation
0,UNITID,root,integer,id,Unit ID for institution
1,INSTNM,school,autocomplete,name,Institution name
2,CITY,school,autocomplete,city,City
3,STABBR,school,string,state,State postcode
4,ACCREDAGENCY,school,string,accreditor,Accreditor for institution


In [52]:
df = pd.read_csv("MERGED2007_08_PP.csv", usecols=cols,na_values=['NU'])

In [57]:
problem_columns = df.iloc[:,-8:-1].columns

In [58]:
for column in problem_columns:
    df[column] = pd.to_numeric(df[column], errors="coerce")

In [59]:
df.describe()

,UNITID,ACCREDAGENCY,HCM2,MAIN,NUMBRANCH,PREDDEG,HIGHDEG,CONTROL,REGION,LOCALE,...,PCTPELL,PCTFLOAN,UG25ABV,COUNT_NWNE_P10,COUNT_WNE_P10,MN_EARN_WNE_P10,MD_EARN_WNE_P10,COUNT_NWNE_P6,COUNT_WNE_P6,MN_EARN_WNE_P6
count,6.890000e+03,0.0,0.0,6890.000000,6890.000000,6890.000000,6890.000000,6890.000000,6890.000000,0.0,...,0.0,0.0,6317.000000,5739.000000,5739.000000,5224.000000,5224.000000,6100.000000,6100.000000,5638.000000
mean,7.409352e+05,NaN,NaN,0.804354,3.328012,1.899129,2.227141,2.109724,4.543977,NaN,...,NaN,NaN,0.398959,174.634257,1310.132427,39527.660796,35747.702910,227.809508,1836.783934,32329.283434
std,3.248851e+06,NaN,NaN,0.396726,9.090477,0.989486,1.286255,0.842947,2.187773,NaN,...,NaN,NaN,0.217234,342.238468,3338.095969,14339.172010,12853.806862,688.950754,5942.791033,11449.700066
min,1.006540e+05,NaN,NaN,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,NaN,...,NaN,NaN,0.000500,0.000000,9.000000,12700.000000,8500.000000,0.000000,3.000000,11000.000000
25%,1.615742e+05,NaN,NaN,1.000000,1.000000,1.000000,1.000000,1.000000,3.000000,NaN,...,NaN,NaN,0.240300,26.000000,144.500000,30100.000000,27500.000000,23.000000,163.000000,24800.000000
50%,2.111185e+05,NaN,NaN,1.000000,1.000000,2.000000,2.000000,2.000000,5.000000,NaN,...,NaN,NaN,0.391700,68.000000,464.000000,36900.000000,34000.000000,65.000000,541.000000,30400.000000
75%,4.043718e+05,NaN,NaN,1.000000,2.000000,3.000000,4.000000,3.000000,6.000000,NaN,...,NaN,NaN,0.545500,174.000000,1111.000000,46300.000000,42600.000000,183.000000,1356.000000,37600.000000
max,4.409890e+07,NaN,NaN,1.000000,74.000000,4.000000,4.000000,3.000000,9.000000,NaN,...,NaN,NaN,1.000000,2466.000000,25672.000000,148400.000000,127400.000000,8098.000000,51176.000000,122400.000000


In [73]:
top_earners = df.sort_values("MN_EARN_WNE_P6")[["INSTNM", "MN_EARN_WNE_P6"]].reset_index().dropna()

In [77]:
top_earners.dropna()[-50:-1]

,index,INSTNM,MN_EARN_WNE_P6
5588,270,California State University Maritime Academy,73200.0
5589,4395,West Virginia School of Osteopathic Medicine,73200.0
5590,3390,Oregon Health & Science University,73500.0
5591,2480,Holy Name Medical Center-Sister Claire Tynan S...,73700.0
5592,4086,The University of Texas Health Science Center ...,74500.0
5593,271,California Western School of Law,74500.0
5594,1799,Babson College,74900.0
5595,412,Los Angeles County College of Nursing and Alli...,75800.0
5596,4202,Eastern Virginia Medical School,75800.0
5597,4113,The University of Texas Health Science Center ...,75900.0
